In [1]:
import pandas as pd

from master_thesis.core.utils.reproducibility import seed_everything
from master_thesis.core.models.llama import load_model_and_tokenizer
from master_thesis.core.utils.prompts import load_prompt
from master_thesis.core.methods.gt_cace import GTCaCEEstimator
from master_thesis.core.utils.reproducibility import save_results


DEVICE = "cuda"
PROMPT_TYPE = "few_shot"
POSITIVE_TOKEN = "true"
NEGATIVE_TOKEN = "false"
MODEL = "LLAMA_2_7B_CHAT"

DATA_DIR = "../../../../data"
DATASETS_DIR = f"{DATA_DIR}/datasets/base_experiments/cebab/test"
CACHE_DIR = f".cache/results"
CLASS_NAMES = ["review_negative", "review_positive"]
LABEL_ASCPECT_NAMES = ["review_negative", "review_positive"]

In [2]:
seed_everything()

In [3]:
model, tokenizer = load_model_and_tokenizer(MODEL)

/home/mateusz_politycki/master-thesis-probing/.venv/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:732: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/mateusz_politycki/master-thesis-probing/.venv/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
TOKENS = [tokenizer.encode(NEGATIVE_TOKEN)[-1], tokenizer.encode(POSITIVE_TOKEN)[-1]]

### Prepare datasets and prompt

In [55]:
test = pd.read_csv(f"{DATASETS_DIR}/test.csv")

In [56]:
prompt = load_prompt(
    DATA_DIR,
    dataset_path="base_experiments/cebab",
    prompt_type=PROMPT_TYPE,
    prompt_aspect="review_aspect",
)

### Estimate GTCace

In [57]:
gt_cace_etimator = GTCaCEEstimator(model, tokenizer, prompt, TOKENS, DEVICE)

In [ ]:
for aspect in ["noise", "food", "ambiance", "service"]:

    CONFOUNDING_ASPECT_NAMES = [f"{aspect}_negative", f"{aspect}_positive"]
    results = gt_cace_etimator.evaluate(test, test, aspect)

    save_results(
        results,
        class_names=CLASS_NAMES,
        aspect_names=CONFOUNDING_ASPECT_NAMES,
        save_path=f"{CACHE_DIR}/gt_cace/{aspect}_aspect.json",
    )